In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("minhhuy2810/rice-diseases-image-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/rice-diseases-image-dataset


In [2]:
!pip install tqdm

In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision import models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, accuracy_score , f1_score , confusion_matrix
from sklearn.preprocessing import StandardScaler 

In [4]:
device= torch.device ( 'cuda' if torch.cuda.is_available() else 'cpu' )

In [5]:
train_transforms= transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]),   
])
val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
])

In [6]:
import os
base_dir = '/kaggle/input/rice-diseases-image-dataset/RiceDiseaseDataset'
train_dir = '/kaggle/input/rice-diseases-image-dataset/RiceDiseaseDataset/train'
val_dir = '/kaggle/input/rice-diseases-image-dataset/RiceDiseaseDataset/validation'

In [7]:
from torchvision.datasets import ImageFolder
train_data = ImageFolder(root=train_dir, transform = train_transforms) 
val_data = ImageFolder( root = val_dir, transform = val_transforms)

In [8]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size = 32 , shuffle = True)
val_loader = DataLoader(val_data, batch_size =32, shuffle = True)

In [9]:
from torchsummary import summary
import torch.nn.functional as F
class CNNModelforRiceDisease(nn.Module):
    def __init__(self,num_classes=4):
        super(CNNModelforRiceDisease , self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels = 32 , kernel_size = 3 , padding = 1 )
        self.conv2 = nn.Conv2d(in_channels=32, out_channels = 64 , kernel_size =3, padding = 1)
        self.conv3 = nn.Conv2d(in_channels=64 , out_channels=128, kernel_size= 3 , padding = 1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels = 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512 , kernel_size=3 , padding=1)
        self.conv6 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3 , padding= 1)
        self.pool = nn.MaxPool2d(kernel_size=2 , stride = 2)
        #Fully Connected Layer
        self.fc1= nn.Linear(1024*3*3, 256)
        self.fc2 = nn. Linear(256, 256)
        self.fc3 = nn. Linear(256, num_classes)

    def forward(self,input_image):
        #Forward pass through convolutional Layers
        input_image=F.leaky_relu(self.conv1(input_image), negative_slope=0.01)
        input_image = self.pool(input_image)
        input_image = F. leaky_relu(self.conv2(input_image), negative_slope=0.01)
        input_image = self.pool(input_image)
        input_image = F. leaky_relu(self.conv3(input_image), negative_slope = 0.01)
        input_image = self.pool(input_image)
        input_image = F. leaky_relu(self.conv4(input_image), negative_slope = 0.01)
        input_image = self.pool(input_image)
        input_image = F. leaky_relu(self.conv5(input_image), negative_slope = 0.01)
        input_image = self.pool(input_image)
        input_image = F. leaky_relu(self.conv6(input_image), negative_slope = 0.01)
        input_image = self.pool(input_image)

        #Flatten for fully connected layers
        input_image = input_image.view(input_image.size(0), -1)

        #Fully Connectedd layers
        input_image = F.leaky_relu(self.fc1(input_image) , negative_slope = 0.01 )
        input_image = F. leaky_relu(self.fc2(input_image), negative_slope= 0.01)
        input_image = self.fc3(input_image)

        return input_image
  
#Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNModelforRiceDisease(num_classes=4).to(device)

#Example to test the movement of tensor to device and check for proper connection with the forward layers
input_tensor = torch.randn(1*3*224*224).to(device)

#summarize the model
summary(model,input_size=(3,224,224))  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
         MaxPool2d-2         [-1, 32, 112, 112]               0
            Conv2d-3         [-1, 64, 112, 112]          18,496
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5          [-1, 128, 56, 56]          73,856
         MaxPool2d-6          [-1, 128, 28, 28]               0
            Conv2d-7          [-1, 256, 28, 28]         295,168
         MaxPool2d-8          [-1, 256, 14, 14]               0
            Conv2d-9          [-1, 512, 14, 14]       1,180,160
        MaxPool2d-10            [-1, 512, 7, 7]               0
           Conv2d-11           [-1, 1024, 7, 7]       4,719,616
        MaxPool2d-12           [-1, 1024, 3, 3]               0
           Linear-13                  [-1, 256]       2,359,552
           Linear-14                  [

In [10]:
#Hyperparameters
batch_size=32
learning_rate=0.01
num_epochs=20

In [11]:
#defining loss functions and optimizers
model= CNNModelforRiceDisease(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= learning_rate)

In [12]:
def train_model(model, train_loader, val_loader , num_epochs=num_epochs, learning_rate=learning_rate):
    train_loss,val_loss=[],[]
    train_accuracy,val_accuracy=[0],[0]
    precision_scores,recall_scores,f1_scores=[],[],[]
    #DEFINE OPTIMIZER & CRITERION
    optimizer = optimizer
    criterion = criterion
    

In [14]:
from tqdm import tqdm
#Initialize metrics
train_loss,val_loss=[],[]
train_accuracy,val_accuracy=[0],[0]
precision_scores,recall_scores,f1_scores=[],[],[]
#Training Loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss,total_train,correct_train = 0.0, 0, 0
    loop = tqdm(train_loader, total= len(train_loader), leave=False)
    for inputs, labels in loop:
        inputs = inputs.to(device)
        labels=labels.to(device)
        #Zero gradients
        optimizer.zero_grad()
        #Forward pass
        output=model(inputs)
        loss=criterion(output,labels)
        #Backward pass and optimization
        loss.backward()
        optimizer.step()
        #update metrics
        epoch_loss = epoch_loss + loss.item()
        _, predicted = torch.max(output, 1)
        correct_train += (predicted==labels).sum().item()
        total_train += labels.size(0)
        #put tqdm description
        loop.set_description(f"Epoch[{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())

    #Training Statistiics
    train_loss.append(epoch_loss/len(train_loader))
    train_accuracy.append(100*correct_train/total_train)
    #Validation loop
    model.eval()
    val_epoch_loss = 0.0
    correct_val,total_val=0,0
    all_labels,all_preds = [],[]

    with torch.no_grad():
        for inputs,labels in val_loader:
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_epoch_loss += loss.item()
            _,predicted = torch.max(outputs,1)
            correct_val += (predicted==labels).sum().item()
            total_val+=labels.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    #Validation metrics
    val_loss.append(val_epoch_loss/ len(val_loader))
    val_accuracy.append(100* correct_val/total_val)
    precision_scores.append(precision_score(all_labels,all_preds,average='weighted', zero_division=0))
    recall_scores.append(recall_score(all_labels,all_preds,average='weighted', zero_division=0))
    f1_scores.append(f1_score(all_labels,all_preds,average='weighted', zero_division=0))

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss[-1]:.4f}, Val Loss:{val_loss[-1]:.4f}, Val Accuracy:{val_accuracy[-1]:.2f}% ")

Epoch 1/20, Train Loss: 5793707.3587, Val Loss:5192224.7031, Val Accuracy:25.00% 


Epoch 2/20, Train Loss: 9723401.0659, Val Loss:545218.8457, Val Accuracy:24.80% 


Epoch 3/20, Train Loss: 977361.1366, Val Loss:377799.3047, Val Accuracy:25.00% 


Epoch 4/20, Train Loss: 232536.0484, Val Loss:356790.0410, Val Accuracy:25.00% 


Epoch 5/20, Train Loss: 143755.0347, Val Loss:49311.6299, Val Accuracy:25.81% 


...

Epoch 16/20, Train Loss: 19924.4491, Val Loss:20233.4999, Val Accuracy:25.00% 


Epoch 17/20, Train Loss: 7081.1512, Val Loss:3646.7214, Val Accuracy:21.95% 


Epoch 18/20, Train Loss: 4086.5503, Val Loss:3979.2845, Val Accuracy:29.67% 


Epoch 19/20, Train Loss: 3790.5749, Val Loss:2265.4185, Val Accuracy:23.58% 


Epoch 20/20, Train Loss: 3449.2874, Val Loss:3881.2581, Val Accuracy:27.44% 
